In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Transjakarta.csv')

In [4]:
# Data filling for corridorID

# 1. Create mappings for corridorID based on corridorName and tapInStopsLat, tapInStopsLon
corridor_name_mapping = df.dropna(subset=['corridorID']).drop_duplicates('corridorName').set_index('corridorName')['corridorID']
corridor_latlon_mapping = df.dropna(subset=['corridorID']).drop_duplicates(['tapInStopsLat', 'tapInStopsLon']).set_index(['tapInStopsLat', 'tapInStopsLon'])['corridorID']

# 2. Apply mappings to fill missing corridorID
df['corridorID'] = df.apply(
    lambda row: corridor_name_mapping.get(row['corridorName'], corridor_latlon_mapping.get((row['tapInStopsLat'], row['tapInStopsLon']), row['corridorID']))
    if pd.isnull(row['corridorID']) else row['corridorID'], axis=1
)


In [3]:
# Create a mapping for corridorName based on unique corridorID
corridor_mapping = df.dropna(subset=['corridorName']).drop_duplicates('corridorID').set_index('corridorID')['corridorName']

# Fill missing corridorName using the mapping
df['corridorName'] = df.apply(
    lambda row: corridor_mapping.get(row['corridorID'], row['corridorName'])
    if pd.isnull(row['corridorName']) else row['corridorName'], axis=1
)

In [4]:
# Create a mapping for tapInStops based on unique tapInStopsLon and tapInStopsLat combinations
tap_in_stops_mapping = df.dropna(subset=['tapInStops']).drop_duplicates(['tapInStopsLon', 'tapInStopsLat']).set_index(['tapInStopsLon', 'tapInStopsLat'])['tapInStops']

# Fill missing tapInStops using the mapping
df['tapInStops'] = df.apply(
    lambda row: tap_in_stops_mapping.get((row['tapInStopsLon'], row['tapInStopsLat']), row['tapInStops'])
    if pd.isnull(row['tapInStops']) else row['tapInStops'], axis=1
)

In [5]:

# 1. Create a mapping for tapOutStops based on unique combinations of tapOutStopsLon and tapOutStopsLat
tap_out_stops_mapping = df.dropna(subset=['tapOutStops']).drop_duplicates(['tapOutStopsLon', 'tapOutStopsLat']).set_index(['tapOutStopsLon', 'tapOutStopsLat'])['tapOutStops']

# 2. Define a function to fill missing tapOutStops using tapOutStopsLon and tapOutStopsLat
def fill_missing_tapOutStops(row):
    return tap_out_stops_mapping.get((row['tapOutStopsLon'], row['tapOutStopsLat']), row['tapOutStops']) if pd.isnull(row['tapOutStops']) else row['tapOutStops']

# 3. Apply the function to fill tapOutStops using tapOutStopsLon and tapOutStopsLat
df['tapOutStops'] = df.apply(fill_missing_tapOutStops, axis=1)


In [6]:
# Create a mapping for tapOutStopsName based on tapOutStops
tap_out_stops_name_mapping = df.dropna(subset=['tapOutStopsName']).drop_duplicates('tapOutStops').set_index('tapOutStops')['tapOutStopsName']

# Fill missing tapOutStopsName using the mapping
df['tapOutStopsName'] = df.apply(
    lambda row: tap_out_stops_name_mapping.get(row['tapOutStops'], row['tapOutStopsName'])
    if pd.isnull(row['tapOutStopsName']) and pd.notnull(row['tapOutStops']) else row['tapOutStopsName'], axis=1
)

In [7]:
# Create a mapping for tapOutStopsLat based on tapOutStops
lat_mapping = df.dropna(subset=['tapOutStopsLat']).drop_duplicates('tapOutStops').set_index('tapOutStops')['tapOutStopsLat']

# Fill missing tapOutStopsLat using the mapping
df['tapOutStopsLat'] = df.apply(
    lambda row: lat_mapping.get(row['tapOutStops'], row['tapOutStopsLat'])
    if pd.isnull(row['tapOutStopsLat']) and pd.notnull(row['tapOutStops']) else row['tapOutStopsLat'], axis=1
)


In [8]:
# Create a mapping for tapOutStopsLon based on tapOutStops
lon_mapping = df.dropna(subset=['tapOutStopsLon']).drop_duplicates('tapOutStops').set_index('tapOutStops')['tapOutStopsLon']

# Fill missing tapOutStopsLon using the mapping
df['tapOutStopsLon'] = df.apply(
    lambda row: lon_mapping.get(row['tapOutStops'], row['tapOutStopsLon'])
    if pd.isnull(row['tapOutStopsLon']) and pd.notnull(row['tapOutStops']) else row['tapOutStopsLon'], axis=1
)


In [9]:
payAmount_20k = pd.DataFrame(df.loc[df['payAmount'] == 20000, 'corridorName'].unique(), columns=['corridorName'])
payAmount_20k

,corridorName
0,Cibubur - Balai Kota
1,Bekasi Barat - Kuningan
2,Palem Semi - Bundaran Senayan
3,Cinere - Bundaran Senayan
4,Bintaro - Fatmawati
5,Rusun Waduk Pluit - Penjaringan
6,Bekasi Barat - Blok M
7,Cinere - Kuningan
8,Cibubur Junction - Blok M
9,Cibubur - Kuningan


In [10]:
# Apply the pay amount logic
df['payAmount'] = df.apply(
    lambda row: 20000 if row['corridorName'] in payAmount_20k and pd.isna(row['payAmount']) 
    else 0 if pd.isna(row['payAmount']) 
    else row['payAmount'], 
    axis=1
)


In [11]:
df.isna().sum()

transID                0
payCardID              0
payCardBank            0
payCardName            0
payCardSex             0
payCardBirthDate       0
corridorID          1257
corridorName           0
direction              0
tapInStops            34
tapInStopsName         0
tapInStopsLat          0
tapInStopsLon          0
stopStartSeq           0
tapInTime              0
tapOutStops         1363
tapOutStopsName     1344
tapOutStopsLat      1344
tapOutStopsLon      1344
stopEndSeq          1344
tapOutTime          1344
payAmount              0
dtype: int64

In [12]:
#drop the rest nan
df = df.dropna()

In [13]:
df.isna().sum()

transID             0
payCardID           0
payCardBank         0
payCardName         0
payCardSex          0
payCardBirthDate    0
corridorID          0
corridorName        0
direction           0
tapInStops          0
tapInStopsName      0
tapInStopsLat       0
tapInStopsLon       0
stopStartSeq        0
tapInTime           0
tapOutStops         0
tapOutStopsName     0
tapOutStopsLat      0
tapOutStopsLon      0
stopEndSeq          0
tapOutTime          0
payAmount           0
dtype: int64

In [14]:
df.shape

(35301, 22)

In [15]:
# Convert 'tapInTime' and 'tapOutTime' to datetime objects
df['tapInTime'] = pd.to_datetime(df['tapInTime'])
df['tapOutTime'] = pd.to_datetime(df['tapOutTime'])

In [16]:
# Convert DataFrame to CSV and Excel files for different uses
df.to_csv('cleaned_Trans.csv', index=False)
df.to_excel('cleaned_Trans.xlsx', index=False, sheet_name='Transjakarta')